In [1]:
%pip install -qqU s3fs

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.37.34 requires botocore<1.38.0,>=1.37.34, but you have botocore 1.37.3 which is incompatible.
s3transfer 0.11.4 requires botocore<2.0a.0,>=1.37.4, but you have botocore 1.37.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [38]:
sft_trainer_config = '''{
    "accelerate_launch_args": {
        "use_fsdp": true,
        "fsdp_version": 2,
        "fsdp_auto_wrap_policy": "TRANSFORMER_BASED_WRAP",
        "fsdp_sharding_strategy": "FULL_SHARD",
        "fsdp_state_dict_type": "SHARDED_STATE_DICT",
        "fsdp_cpu_ram_efficient_loading": true
      },
    "model_name_or_path": "meta-llama/Llama-4-Scout-17B-16E-Instruct",
    "training_data_path": "epfl-dlab/gsm8k",
    "validation_data_path": "epfl-dlab/gsm8k",
    "output_dir": "/mnt/output/model",
    "save_model_dir": "/mnt/output/model",
    "num_train_epochs": 3.0,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "gradient_checkpointing": true,
    "save_strategy": "epoch",
    "learning_rate": 1e-5,
    "weight_decay": 0.0,
    "lr_scheduler_type": "cosine",
    "include_tokens_per_second": true,
    "data_formatter_template": "### Question:{{question}} \\n\\n### Answer: {{answer}}",
    "response_template": "\\n### Answer:",
    "use_flash_attn": false,
    "fast_moe": true,
    "torch_dtype": "bfloat16",
    "max_seq_length": 4096,
    "peft_method": "lora",
    "r": 8,
    "lora_dropout": 0.05,
    "lora_alpha": 32,
    "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"],
    "lora_post_process_for_vllm": true
}'''

In [39]:
# Convert training configuration to base64 representation to pass it using environment variable
import base64


def encode_config(config):
    base64_bytes = base64.b64encode(config.encode("ascii"))
    txt = base64_bytes.decode("ascii")
    return txt

encoded_config = encode_config(sft_trainer_config)

In [40]:
from kubeflow.training import TrainingClient
from kubernetes.client import (
    V1EnvVar,
    V1EnvVarSource,
    V1SecretKeySelector,
    V1Volume,
    V1VolumeMount,
    V1PersistentVolumeClaimVolumeSource
)

job_name = "llama4"

tc = TrainingClient()

tc.create_job(
    job_kind="PyTorchJob",
    name=job_name,
    num_workers=1,
    num_procs_per_worker="auto",
    resources_per_worker={"gpu": 8},
    base_image="image-registry.openshift-image-registry.svc:5000/fms-hf-tuning/fms-hf-tuning:2.8.2",
    env_vars=[
        V1EnvVar(name="HF_TOKEN", value_from=V1EnvVarSource(secret_key_ref=V1SecretKeySelector(key="HF_TOKEN", name="hf-token"))),
        V1EnvVar(name="HF_HOME", value="/mnt/scratch/hf_home"),
        V1EnvVar(name="SFT_TRAINER_CONFIG_JSON_ENV_VAR", value=encoded_config),
    ],
    volumes=[
        V1Volume(name="trained-model",
                 persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name="trained-model")),
        V1Volume(name="scratch-volume",
                 persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name="scratch-volume")),
    ],
    volume_mounts=[
        V1VolumeMount(name="trained-model", mount_path="/mnt/output"),
        V1VolumeMount(name="scratch-volume", mount_path="/mnt/scratch"),
    ],
)

In [ ]:
logs, _ = tc.get_job_logs(job_name, follow=True)

[Pod llama4-master-0]: WARNING:accelerate.commands.launch:The following values were not passed to `accelerate launch` and had defaults used instead:
[Pod llama4-master-0]: 	`--num_machines` was set to a value of `1`
[Pod llama4-master-0]: 	`--mixed_precision` was set to a value of `'no'`
[Pod llama4-master-0]: 	`--dynamo_backend` was set to a value of `'no'`
[Pod llama4-master-0]: To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[Pod llama4-master-0]: WARNING:sft_trainer.py:You are running lora with the ScatterMoE plugin, please note that passing target modules that are part of the moe module can cause unexpected behaviors and unsuccessful tuning while LoRA tuning with ScatterMoE. For safe tuning, only pass linear modules such as those in the attn layer (i.e. ['q_proj', 'v_proj', 'o_proj', 'k_proj'])
Fetching 50 files:   0%|          | 0/50 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not instal

In [2]:
# Upload trained LoRA layer to the S3 bucket containing base model
import s3fs
import os


s3 = s3fs.S3FileSystem(
      key=os.environ["AWS_ACCESS_KEY_ID"],
      secret=os.environ["AWS_SECRET_ACCESS_KEY"],
      endpoint_url=os.environ["AWS_S3_ENDPOINT"]
   )
s3_path = os.environ["AWS_S3_BUCKET"] + "/meta-llama/Llama-4-Scout-17B-16E-Instruct/adapter"
_ = s3.put("/opt/app-root/src/model", s3_path, recursive=True)